 <h3> Partições </h3>

<b> imports </b>

In [3]:
from pyspark.sql import SparkSession

In [9]:
from pyspark.sql import functions as F

In [ ]:
from pyspark.sql.functions import *

<b> load session </b>

In [11]:
sparkContext = (SparkSession
                .builder
                .master('local[*]')
                .appName("minhas partições")
                .getOrCreate())

<b> load csv </b>

In [37]:
url = "/Users/macca/Documents/spark/pyspark_com_grimaldo/arquivo_de_dados/Bikes.csv"

In [40]:
df2 = (sparkContext
      .read
      .format("csv")
      .option("header", "true")
      .option("inferSchema", "true")
      .option("delimiter", ",")
     .load(url))

<b> numero de partições </b>

In [79]:
df2.rdd.getNumPartitions()

1

In [80]:
df2.show(10)

+--------------------+--------+---------+----------+------------+---+-----+-------------+
|           bike_name|   price|     city|kms_driven|       owner|age|power|        brand|
+--------------------+--------+---------+----------+------------+---+-----+-------------+
|TVS Star City Plu...| 35000.0|Ahmedabad|   17654.0| First Owner|3.0|110.0|          TVS|
|Royal Enfield Cla...|119900.0|    Delhi|   11000.0| First Owner|4.0|350.0|Royal Enfield|
|Triumph Daytona 675R|600000.0|    Delhi|     110.0| First Owner|8.0|675.0|      Triumph|
|TVS Apache RTR 180cc| 65000.0|Bangalore|   16329.0| First Owner|4.0|180.0|          TVS|
|Yamaha FZ S V 2.0...| 80000.0|Bangalore|   10000.0| First Owner|3.0|150.0|       Yamaha|
|    Yamaha FZs 150cc| 53499.0|    Delhi|   25000.0| First Owner|6.0|150.0|       Yamaha|
|Honda CB Hornet 1...| 85000.0|    Delhi|    8200.0| First Owner|3.0|160.0|        Honda|
|Hero Splendor Plu...| 45000.0|    Delhi|   12645.0| First Owner|3.0|100.0|         Hero|
|Royal Enf

In [58]:
#df_repartition = df2.repartition(8)

<b> inicia Sql Server </b>

In [82]:
df2.createOrReplaceTempView("tbdelhi")

In [91]:
df3 = sparkContext.sql("select * from tbdelhi where city='Delhi'")

In [96]:
df3.show(10)

+--------------------+--------+-----+----------+------------+---+-----+-------------+
|           bike_name|   price| city|kms_driven|       owner|age|power|        brand|
+--------------------+--------+-----+----------+------------+---+-----+-------------+
|Royal Enfield Cla...|119900.0|Delhi|   11000.0| First Owner|4.0|350.0|Royal Enfield|
|Triumph Daytona 675R|600000.0|Delhi|     110.0| First Owner|8.0|675.0|      Triumph|
|    Yamaha FZs 150cc| 53499.0|Delhi|   25000.0| First Owner|6.0|150.0|       Yamaha|
|Honda CB Hornet 1...| 85000.0|Delhi|    8200.0| First Owner|3.0|160.0|        Honda|
|Hero Splendor Plu...| 45000.0|Delhi|   12645.0| First Owner|3.0|100.0|         Hero|
|Royal Enfield Cla...| 88000.0|Delhi|   19000.0|Second Owner|7.0|500.0|Royal Enfield|
| Bajaj Discover 100M| 29499.0|Delhi|   20000.0| First Owner|8.0|100.0|        Bajaj|
| Bajaj Discover 125M| 29900.0|Delhi|   20000.0| First Owner|7.0|125.0|        Bajaj|
| Bajaj Discover 125M| 29900.0|Delhi|   20000.0| First

In [97]:
df4 = df3.filter(df3.owner.isNotNull())

<b> Grava particionado por owner </b>

In [88]:
url3 = "/Users/macca/Documents/spark/pyspark_com_grimaldo/arquivo_de_dados/delhi"

In [98]:
df4.write.option("header",True).partitionBy("owner").mode("overwrite").csv(url3)

<b> Recupera partição </b>

In [99]:
df5=sparkContext.read.option("header",True).csv(url3 + "/owner=First Owner")

In [100]:
df5.show()

+--------------------+--------+-----+----------+----+-----+-------------+
|           bike_name|   price| city|kms_driven| age|power|        brand|
+--------------------+--------+-----+----------+----+-----+-------------+
|Royal Enfield Cla...|119900.0|Delhi|   11000.0| 4.0|350.0|Royal Enfield|
|Triumph Daytona 675R|600000.0|Delhi|     110.0| 8.0|675.0|      Triumph|
|    Yamaha FZs 150cc| 53499.0|Delhi|   25000.0| 6.0|150.0|       Yamaha|
|Honda CB Hornet 1...| 85000.0|Delhi|    8200.0| 3.0|160.0|        Honda|
|Hero Splendor Plu...| 45000.0|Delhi|   12645.0| 3.0|100.0|         Hero|
| Bajaj Discover 100M| 29499.0|Delhi|   20000.0| 8.0|100.0|        Bajaj|
| Bajaj Discover 125M| 29900.0|Delhi|   20000.0| 7.0|125.0|        Bajaj|
| Bajaj Discover 125M| 29900.0|Delhi|   20000.0| 7.0|125.0|        Bajaj|
|Hero Splendor iSm...| 46500.0|Delhi|    3500.0| 2.0|110.0|         Hero|
|Yamaha FZ V 2.0 1...| 45000.0|Delhi|   23000.0| 6.0|150.0|       Yamaha|
|Royal Enfield Cla...| 85000.0|Delhi| 